# Esercitazione 5: Language Model con RNN

In questa esercitazione vedremo come utilizzare un **language model** basato su RNN per generare nomi di supereroi.



### Dati
 Come primo step analizzamo i dati. Come sorgente di dati per il training della RNN è stato utilizzato il [Superhero Names Dataset](https://github.com/am1tyadav/superhero), una dataset contenente una lista di 9000+ nomi di supereroi e supercattivi.

In [1]:
from pathlib import Path
import tensorflow as tf
import numpy as np

BEGIN_NAME_TOKEN= '$'
END_NAME_TOKEN='\t'
PAD_CODE = 0

with Path('data/superheroes.txt').open() as file:
  data = file.readlines()

data[:5]

['jumpa\t\n', 'doctor fate\t\n', 'starlight\t\n', 'isildur\t\n', 'lasher\t\n']

Come si può notare dall'output, in ogni riga abbiamo un nome delimitato dal token "\t" per indicare la fine della stringa. Manca però il token che delimita l'**inizio** del nome, dunque effettuiamo un breve processamento del file per aggiungere anche il carattere di inizio stringa:

In [2]:
data_with_begin_token = [f"{BEGIN_NAME_TOKEN}{line}" for line in data] # add begin token

with Path('data/superheroes_preprocessed.txt').open('w') as file:
  for line in data_with_begin_token:
    file.writelines(line)

with Path('data/superheroes_preprocessed.txt').open() as file:
  data = file.read()
  
data.splitlines()[:5]

['$jumpa\t', '$doctor fate\t', '$starlight\t', '$isildur\t', '$lasher\t']

### Build a char level vocabulary

La generazione dei nomi avviene carattere-per-carattere, dunque dobbiamo tokenizzare i dati in input a livello di carattere e costruire un vocabolario che rappresenta un **indice** per i token individuati.

Inoltre le reti NN prendono in input solo dati numerici, dobbiamo dunque associare ad ogni carattere del vocabolario un indice intero. Per tenere traccia del mapping tra caratteri e indici numerici, utilizzamo una semplice namedtuple come struttura dati: `Vocabulary`.

In [3]:
from collections import namedtuple

Vocabulary = namedtuple("Vocabulary", ['char_to_index', 'index_to_char', 'size'])

In [4]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    filters='!"#%&()*+,-.:;/<=>?@[\\]^_`{|}~', # remove punctuations but retain $, \t and \n
    split='\n',
)

tokenizer.fit_on_texts(data)

vocab = Vocabulary(char_to_index = tokenizer.word_index, 
                   index_to_char = tokenizer.index_word,
                   size=len(tokenizer.index_word))

In [5]:
for char_entry, idx_entry in zip(vocab.char_to_index.items(), vocab.index_to_char.items()):
    print(f"{char_entry} <----> {idx_entry}")

('$', 1) <----> (1, '$')
('\t', 2) <----> (2, '\t')
('a', 3) <----> (3, 'a')
('e', 4) <----> (4, 'e')
('r', 5) <----> (5, 'r')
('o', 6) <----> (6, 'o')
('n', 7) <----> (7, 'n')
('i', 8) <----> (8, 'i')
(' ', 9) <----> (9, ' ')
('t', 10) <----> (10, 't')
('s', 11) <----> (11, 's')
('l', 12) <----> (12, 'l')
('m', 13) <----> (13, 'm')
('h', 14) <----> (14, 'h')
('d', 15) <----> (15, 'd')
('c', 16) <----> (16, 'c')
('u', 17) <----> (17, 'u')
('g', 18) <----> (18, 'g')
('k', 19) <----> (19, 'k')
('b', 20) <----> (20, 'b')
('p', 21) <----> (21, 'p')
('y', 22) <----> (22, 'y')
('w', 23) <----> (23, 'w')
('f', 24) <----> (24, 'f')
('v', 25) <----> (25, 'v')
('j', 26) <----> (26, 'j')
('z', 27) <----> (27, 'z')
('x', 28) <----> (28, 'x')
('q', 29) <----> (29, 'q')


Sebbene la generazione dei nomi avviene a livello di singoli caratteri, l'input della rete sono sequenze di indici (che rappresentano caratteri).

Le funzioni `name_to_seq` e `seq_to_name`, effettuano questa conversione da stringa di caratteri a sequenza di indici, e viceversa. 



In [6]:
def name_to_seq(name, tokenizer):
  return np.array(tokenizer.texts_to_sequences(name)).ravel() # flatten list of list into list of int

def seq_to_name(seq, tokenizer):
    return ''.join([tokenizer.index_word[idx] for idx in seq if idx != 0])

for name in data.splitlines()[:10]:
  seq = name_to_seq(name, tokenizer)
  name_from_seq = seq_to_name(seq, tokenizer)
  print(f"{name} --> {seq} --> {name_from_seq}")

$jumpa	 --> [ 1 26 17 13 21  3  2] --> $jumpa	
$doctor fate	 --> [ 1 15  6 16 10  6  5  9 24  3 10  4  2] --> $doctor fate	
$starlight	 --> [ 1 11 10  3  5 12  8 18 14 10  2] --> $starlight	
$isildur	 --> [ 1  8 11  8 12 15 17  5  2] --> $isildur	
$lasher	 --> [ 1 12  3 11 14  4  5  2] --> $lasher	
$varvara	 --> [ 1 25  3  5 25  3  5  3  2] --> $varvara	
$the target	 --> [ 1 10 14  4  9 10  3  5 18  4 10  2] --> $the target	
$axel	 --> [ 1  3 28  4 12  2] --> $axel	
$battra	 --> [ 1 20  3 10 10  5  3  2] --> $battra	
$changeling	 --> [ 1 16 14  3  7 18  4 12  8  7 18  2] --> $changeling	


### Preparazione del Dataset di train
Come già accennato in precedenza, l'input della rete neurale è una sequenza e l'output un distribuzione di probabilità (sul vocabolario) **condizionata sulla sequenza in input**.

Di seguito generiamo sequenze incrementeali che costituiranno l'input della rete

In [7]:
sequences = []
names = data.splitlines()
for name in names:
    seq = name_to_seq(name, tokenizer)
    if len(seq) >= 2: # minimal length for a seq
      # build new seq incrementtaly shifting to the right one by one
      sequences += [seq[:i] for i in range(2, len(seq) +1)] 

In [8]:
for sequence in sequences[:15]:
  name = seq_to_name(sequence, tokenizer)
  print(f"{sequence} --> {name}")

[ 1 26] --> $j
[ 1 26 17] --> $ju
[ 1 26 17 13] --> $jum
[ 1 26 17 13 21] --> $jump
[ 1 26 17 13 21  3] --> $jumpa
[ 1 26 17 13 21  3  2] --> $jumpa	
[ 1 15] --> $d
[ 1 15  6] --> $do
[ 1 15  6 16] --> $doc
[ 1 15  6 16 10] --> $doct
[ 1 15  6 16 10  6] --> $docto
[ 1 15  6 16 10  6  5] --> $doctor
[ 1 15  6 16 10  6  5  9] --> $doctor 
[ 1 15  6 16 10  6  5  9 24] --> $doctor f
[ 1 15  6 16 10  6  5  9 24  3] --> $doctor fa


Il problema principale nell'aver generato le sequenze in questo modo è che la rete accetta solo **input a dimensione fissa**. Dunque dobbiamo effettuare il **padding** delle sequenze in modo tale da rendere la lunghezza delle sequenze uniforme.

In [9]:
max_seq_len = max([len(seq) for seq in sequences]) # needed for padding
padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_seq_len, value=PAD_CODE)

In [10]:
padded_sequences.shape

(97332, 34)

Come si può notare la lunghezza delle sequenze è costante (34).

Ultimo step è quello di dividere il dataset in train e test set. Prima però, ricordiamo che siamo in un contesto di **supervised learning** e dunque necessitiamo di **labels** da associare all'input in modo tale che la rete possa calcolare la *loss function**.

Per soddisfare questo requisito, utilizziamo una strategia chiamata **Teacher Forcing**. 

In [11]:
from sklearn.model_selection import train_test_split

X, y = (padded_sequences[:,:-1], # exclude last char of the sequence
       padded_sequences[:,-1]) # the label is just the last char 
       
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=1990)

In [12]:
print(f"Dataset shape: {X.shape}, labels shape {y.shape}")

Dataset shape: (97332, 33), labels shape (97332,)


### Specifica dell'architettura della RNN

L'architettura scelta è molto semplice, un livello di **embdedding** che mappa ogni token del vocabolario in uno spazio vettoriale multidimensionale ed un **livello hidden LSTM**. Il livello di output ha una funzione di attivazioe *+soft-max** che restituisce una distribuzione di probabilità sull'intero vocabolario.

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense
from collections import namedtuple


hyperparams = {
    'vocab_size': vocab.size + 1 , # +1 is due to the padding char
    'embedding_size': 16,
    'seq_len': max_seq_len -1, # -1 since we split the last char of the seq for the label
    'keep_prob': 0.7,
    'hidden_units': 64,
    'unroll': True,
    'loss': 'sparse_categorical_crossentropy',
    'optimizer': 'adam',
    'batch_size': 128,
    'epochs': 100,
}
HyperParams = namedtuple("hyperparameters", hyperparams)
HP = HyperParams(**hyperparams)


model = Sequential(name='LM-RNN')

model.add(Embedding(HP.vocab_size, HP.embedding_size, input_length=HP.seq_len))
model.add(Dropout(1-HP.keep_prob))
model.add(LSTM(HP.hidden_units))
model.add(Dense(HP.vocab_size, activation='softmax'))

model.compile(loss=HP.loss, optimizer=HP.optimizer, 
              metrics=['accuracy'])

model.summary()

Model: "LM-RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 33, 16)            480       
_________________________________________________________________
dropout (Dropout)            (None, 33, 16)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 30)                1950      
Total params: 23,166
Trainable params: 23,166
Non-trainable params: 0
_________________________________________________________________


### Training

In [14]:
import tensorflow.keras as keras

class MakeNamesCallback(keras.callbacks.Callback):
    def __init__(self, tokenizer, n_names=10):
      super().__init__()
      self.tokenizer = tokenizer
      self.n_names = n_names
 
    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())

        for i in range(0, self.n_names):
          name = self.generate_name(self.tokenizer, self.model)
          print(f"Generated name {name}")

    def generate_name(self, tokenizer, model, seed=BEGIN_NAME_TOKEN):
        """greedy search"""
        name = seed
        for i in range(0, 40):
          seq = name_to_seq(name, tokenizer)
          padded_seq = tf.keras.preprocessing.sequence.pad_sequences([seq], padding='pre',
                                                                    maxlen=max_seq_len-1,value=PAD_CODE)
        
          prediction = model.predict(padded_seq)[0]
          p = prediction[1:]/prediction[1:].sum()
          index_vocab = list(tokenizer.index_word.keys())
          best_idx = np.random.choice(index_vocab, p=p) # the 0-index is the softmax output for the pad character
          best_char = tokenizer.index_word[best_idx]

          name += best_char

          if best_char == END_NAME_TOKEN:
            break
        return name

In [15]:
history = model.fit(X_train, y_train, 
          batch_size=HP.batch_size, epochs=HP.epochs,
          validation_data=(X_test, y_test), verbose=2,
          callbacks=[MakeNamesCallback(tokenizer=tokenizer)])

#model.save(f"{model.name}")

Epoch 1/100
Generated name $dstkjmhjtpyeilvlnjrm	
Generated name $ph nvuclwm$uaaxpxqgo	
Generated name $pwfaej$ovsbtnayrymnh sxqgnrrtlpieuh$mt k
Generated name $xglgnqoy 	
Generated name $gkgznuonlijpyk g	
Generated name $czfwyoflbehqlmw cmk xs$hgzeefmav tgeyntz
Generated name $v$eybf ctm $mpttatpuuuq	
Generated name $ootnojz vvpvfxrtftfmvpnq$c$jiyyxtitxmqwx
Generated name $ ozrlie cvmgllym$ rmmtrosmx	
Generated name $ldapq	
609/609 - 48s - loss: 2.8657 - accuracy: 0.1610 - val_loss: 2.6625 - val_accuracy: 0.2045
Epoch 2/100
Generated name $aymi	
Generated name $ullactsdir	
Generated name $meosigmigoor	
Generated name $remeonsihel	
Generated name $ore jamte	
Generated name $hosliylrnke	
Generated name $reecivar	
Generated name $jtatdacircybachi	
Generated name $mehwy bdlemearh	
Generated name $lamhe ern mabgant	
609/609 - 12s - loss: 2.6264 - accuracy: 0.2103 - val_loss: 2.5707 - val_accuracy: 0.2255
Epoch 3/100
Generated name $olemas	
Generated name $tyaropi	
Generated name $boomdkim	

### Generazione dei Nomi

Sono state definite due funzioni con differenti strategie per la generazione dei nomi:

* 'generate_name_from_seed': dato un seed iniziale, ovvero una sequenza di caratteri, l'i-esimo carattere $c_i$ generato sarà quello che massimizza la probabilità:

$$ c_i = \operatorname*{argmax}_i P(c_i | c_{i-1},\ldots, c_0) $$

* 'generate_name_greedy': dato un seed iniziale, ovvero una sequenza di caratteri, l'i-esimo carattere $c_i$ generato è dato da una v.a.

$$ C \sim P(c_i | c_{i-1},\ldots, c_0) $$

Questa seconda implementazione, è la versione non-deterministica della prima, in quanto ad ogni step viene campionato dal vocabolario un carattere secondo la distribuzione definita dalla rete (l'output della softmax).


In [ ]:
model = keras.models.load_model('data/LM-RNN-model')

In [ ]:
def generate_name_from_seed(tokenizer, model, seed=BEGIN_NAME_TOKEN):
  name = seed
  for i in range(0, 50):
    seq = name_to_seq(name, tokenizer)
    #print(seq)
    padded_seq = tf.keras.preprocessing.sequence.pad_sequences([seq], padding='pre',
                                                              maxlen=max_seq_len-1,value=PAD_CODE)
  
    prediction = model.predict(padded_seq)[0]
    best_idx = tf.argmax(prediction).numpy()
    best_char = tokenizer.index_word[best_idx]
    #print(best_char)
    name += best_char

    if best_char == END_NAME_TOKEN:
      break
  return name

def generate_name_greedy(tokenizer, model, seed=BEGIN_NAME_TOKEN):
  name = seed
  for i in range(0, 50):
    seq = name_to_seq(name, tokenizer)
    padded_seq = tf.keras.preprocessing.sequence.pad_sequences([seq], padding='pre',
                                                              maxlen=max_seq_len-1,value=PAD_CODE)
  
    prediction = model.predict(padded_seq)[0]
    index_vocab = list(tokenizer.index_word.keys())
    best_idx = np.random.choice(index_vocab, p=prediction[1:]) # the 0-index is the softmax output for the pad character
    best_char = tokenizer.index_word[best_idx]

    name += best_char

    if best_char == END_NAME_TOKEN:
      break
  return name

In [ ]:
import itertools

# generate names from permutation constructed seeds
for i in range(1,3):
  for perm in itertools.permutations("aemo",i):
    seed = "".join(perm)
    name = generate_name_from_seed(tokenizer,model, seed=seed)
    print(name)

aranno	
ewhor black	
maracan	
owson man	
aeron man	
ampha sand	
aomed	
eara	
emparder sand	
eosed grey	
maracan	
mestar black	
moss black cat	
oard boy	
oed man man	
ommer the sparn spider	


In [ ]:
for i in range(25):
    name = generate_name_greedy(tokenizer, model, seed=BEGIN_NAME_TOKEN)
    print(name)

$knini burtie	
$ragus eld woor	
$brtera	
$megomankam javo	
$rosaon	
$coldepsi achora	
$thon xuka	
$sharm	
$cyrsentis	
$krowpler	
$tompon skyven	
$spodon	
$the mordiss	
$rig demon	
$spurk roy	
$spy	
$flif gillawk	
$superman	
$elon cara	
$jesse shaho	
$genseor	
$packmon bwutiter	
$maxs luctor	
$arico	
$yeldshoy	


### Risultati

* è interessante notare come alcuni dei nomi generati, sopratutto superato il quarto delle epoche, presentino delle features tipiche dei nomi di supereroi: il finale "man", la particella "the", i prefissi "dr" e "mr", fenomeno che segnala come la rete effettivamente stia apprendendo particolari pattern presenti nel dataset.

* In alcuni casi si possono ritrovare nomi generati di veri supereroi "superman", "hulk", "ultron", "thor" fenomeno (forse) indicativo di un possibile overfitting.

* Anche nelle epoche finali, vengono comunque generati nomi "spuri" (es. "$pwofessor phayeriex") che chiaramente rappresentano errori nell'apprendimento della distribuzione.